In [1]:
import pandas as pd

# read in training data, edge data, and initialize NUM_DELAYS
data_df = pd.read_csv('./train_data.csv')
edges_df = pd.read_csv('./edges.csv')
NUM_DELAYS = data_df.shape[0]
nodes_with_parents = list(set(edges_df.Target))
nodes_no_parents = list(set(edges_df.Source) - set(edges_df.Target))
nodes_set = set(nodes_with_parents + nodes_no_parents)

To compute the conditional probability tables, we will obesrve each node in a BFS-like fashion, starting with nodes that do not have parents and then working our way through nodes whose parents we have already observed.

In [2]:
# make dictionary of nodes and their parents
node_parents = dict()
nodes_with_parents
for node in nodes_with_parents:
    node_parents[node] = set(edges_df.Source[edges_df.Target == node])
for node in nodes_no_parents:
    node_parents[node] = set()
    
# keep track of a visted set
visited = set()

# Nodes Without Parents

In [3]:
print("These are the list of parentless nodes :", nodes_no_parents,". We start by defining a function to reduce redundancy.")

These are the list of parentless nodes : ['MONTH', 'DAY_OF_WEEK', 'SECURITY_DELAY'] . We start by defining a function to reduce redundancy.


In [4]:
def parentless(col):
    '''
    Find probability tables of nodes which do not have parents.
    '''
    df = data_df.groupby(col).count()['QUARTER'].to_frame()
    df.columns = ['count']
    df['probability'] = df['count'] / NUM_DELAYS
    df.reset_index(level=0, inplace=True)
    df.columns = [col,'count','probability']
    visited.add(col)
    nodes_set.remove(col)
    return df

## 1. Month

In [5]:
month_df = parentless('MONTH')
month_df

MONTH  count  probability
0       1     53     0.068475
1       2     27     0.034884
2       3     41     0.052972
3       4     45     0.058140
4       5     60     0.077519
5       6     84     0.108527
6       7     90     0.116279
7       8     86     0.111111
8       9     96     0.124031
9      10     74     0.095607
10     11     53     0.068475
11     12     65     0.083979

## 2. Day of Week

In [6]:
day_of_week_df = parentless('DAY_OF_WEEK')
day_of_week_df

DAY_OF_WEEK  count  probability
0            1    126     0.162791
1            2    100     0.129199
2            3    100     0.129199
3            4    134     0.173127
4            5    133     0.171835
5            6     75     0.096899
6            7    106     0.136951

## 3. Security Delay

In [7]:
security_delay_df = parentless('SECURITY_DELAY')
security_delay_df

SECURITY_DELAY  count  probability
0             0.0    773     0.998708
1             1.0      1     0.001292

# Nodes with Parents

First, we find all the nodes whose values we can compute based on those nodes whose conditional probability tables we have already computed.

In [8]:
def available_nodes():
    li = []
    for node in nodes_set:
        if node_parents[node].issubset(visited):
            li += [node]
    return li

In [9]:
available_nodes()

['NAS_DELAY', 'QUARTER', 'ScheduledArrivalPHL']

## 4. Quarter

In [10]:
temp_df = data_df.groupby(list(node_parents['QUARTER'])).count()['QUARTER'].to_frame()
temp_df

QUARTER
MONTH DAY_OF_WEEK         
1     1                 11
      2                  5
      3                  9
      4                  9
      5                  6
      6                  6
      7                  7
2     1                  6
      2                  3
      3                  5
      4                  7
      5                  5
      6                  1
3     1                  7
      2                  2
      3                  6
      4                  7
      5                  8
      6                  3
      7                  8
4     1                  7
      2                  4
      3                  6
      4                  8
      5                  7
      6                  9
      7                  4
5     1                 12
      2                  8
      3                  7
...                    ...
8     6                  5
      7                  5
9     1                 13
      2                 12
      3                 10
      4                 16
      5                 20
      6                 10
      7                 15
10    1                 15
      2                  8
      3                 10
      4                 14
      5                 10
      6                  7
      7                 10
11    1                  7
      2                  6
      3                  4
      4                  8
      5                  8
      6                  5
      7                 15
12    1                 10
      2                 12
      3                  7
      4                 12
      5                 12
      6                  2
      7                 10

[83 rows x 1 columns]

In [11]:
crs_dep_df = parentless('CRS_DEP_TIME')
crs_dep_df

CRS_DEP_TIME  count  probability
0             0    438     0.565891
1             1    336     0.434109

## 4. CRS Arrival Time

In [12]:
crs_arr_df = parentless('CRS_ARR_TIME')
crs_arr_df

CRS_ARR_TIME  count  probability
0             0    299     0.386305
1             1    475     0.613695

## 6. Number of Scheduled Departures at SFO When Delayed Flight was to Depart

0 represents less than 10, 1 represents less than 20, 2 represents less than 30, 3 represents less than 40, 4 represents less than 50.

In [13]:
scheduled_departure_df = parentless('ScheduledDepartureSFO')
scheduled_departure_df

ScheduledDepartureSFO  count  probability
0                    0.0    571     0.737726
1                    1.0     19     0.024548
2                    2.0     23     0.029716
3                    3.0     72     0.093023
4                    4.0     89     0.114987

## 7. Number of Scheduled Arrivals at PHL When Delayed Flight was to Arrive

0 represents less than 10, 1 represents less than 20, 2 represents less than 30, 3 represents less than 40, 4 represents less than 50.

In [14]:
scheduled_arrival_df = parentless('ScheduledArrivalPHL')
scheduled_arrival_df

ScheduledArrivalPHL  count  probability
0                  0.0    520     0.671835
1                  1.0    172     0.222222
2                  2.0     45     0.058140
3                  3.0     10     0.012920
4                  4.0     27     0.034884